In [48]:
import pandas as pd
import numpy as np
import pickle
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix, save_npz, load_npz
from implicit.als import AlternatingLeastSquares

In [42]:
df = pd.read_csv("../data/processed/electronics_subset.csv")
df = df[
    ["reviewerID", "asin", "overall", "reviewText", "unixReviewTime"]
]
df["reviewText"] = df["reviewText"].fillna("").astype(str)
df.head()

,reviewerID,asin,overall,reviewText,unixReviewTime
0,A3D6FSS8RJFDE8,B006K551TO,3,I purchased this camera because we have a Sony...,1374624000
1,A3SO3G35DXC95Q,B000YB2IPK,4,I purchased this player a few days ago. I deci...,1201910400
2,A1CTVZ3W8XZWZ9,B00FXYT12G,5,Nikon DSLR cameras (with an APS-C sensor) have...,1395619200
3,AJDK7OUKLYX8,B0039UWW54,4,Modem works great but be prepared to have a ha...,1335139200
4,A1172I8JR1L2P6,B00D8JA2S0,5,"Mediabridge cables always work well for me, an...",1390780800


In [43]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df["user_id"] = user_encoder.fit_transform(df["reviewerID"])
df["item_id"] = item_encoder.fit_transform(df["asin"])
df[["reviewerID", "user_id", "asin", "item_id"]].head()

,reviewerID,user_id,asin,item_id
0,A3D6FSS8RJFDE8,109231,B006K551TO,43688
1,A3SO3G35DXC95Q,129075,B000YB2IPK,13629
2,A1CTVZ3W8XZWZ9,16600,B00FXYT12G,58991
3,AJDK7OUKLYX8,153373,B0039UWW54,27052
4,A1172I8JR1L2P6,1613,B00D8JA2S0,56985


In [44]:
interactions = coo_matrix(
    (df["overall"], (df["user_id"], df["item_id"]))
).tocsr()
interactions.shape, interactions.nnz

((174494, 60375), 500000)

In [45]:
def compute_sparsity(X):
    return 1 - (X.nnz / (X.shape[0] * X.shape[1]))
print("Interactions sparsity:", compute_sparsity(interactions))

Interactions sparsity: 0.9999525394942006


In [49]:
train = interactions.copy().tolil()
test_items = {}
for user in range(train.shape[0]):
    items = train.rows[user]
    if len(items) > 1:
        test_items[user] = items[-1]
        train[user, items[-1]] = 0
train = train.tocsr()
print("Train sparsity:", compute_sparsity(train))

Train sparsity: 0.9999649718686158


In [50]:
als_model = AlternatingLeastSquares(
    factors=50,
    regularization=0.1,
    iterations=20,
    random_state=42
)
als_model.fit(train)

C:\Users\Sriraj Yamana\AppData\Local\Programs\Python\Python310\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

In [51]:
def precision_at_k_test(model, train, test_items, k=10):
    hits = 0
    for user, true_item in test_items.items():
        rec_items, _ = model.recommend(
            user,
            train[user],
            N=k
        )
        if true_item in rec_items:
            hits += 1
    return hits / len(test_items)
precision_at_k_test(als_model, train, test_items, k=10)

0.0037640483752748597

In [52]:
os.makedirs("../models", exist_ok=True)
os.makedirs("../data/processed", exist_ok=True)
pickle.dump(als_model, open("../models/als_model.pkl", "wb"))
pickle.dump(user_encoder, open("../models/user_mapping.pkl", "wb"))
pickle.dump(item_encoder, open("../models/item_mapping.pkl", "wb"))
save_npz("../data/processed/interactions.npz", interactions)
save_npz("../data/processed/train.npz", train)

In [53]:
print(interactions.shape)
print(train.shape)
print(interactions.nnz, train.nnz)

(174494, 60375)
(174494, 60375)
500000 369024
